# Preprocessing and EDA

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col, when, count, trim, avg, round as spark_round, 
    to_date, split, size, substring
)
from pyspark.sql import Window
import os

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("NOAA_Parquet_Processor") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

base_path = "/home/alumno/reposirotio/Grupo3"
path_2025 = f"{base_path}/2025.tar/*"

25/12/31 12:59:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.types import *

schema = StructType([
    StructField("STATION", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("LATITUDE", DoubleType(), True),
    StructField("LONGITUDE", DoubleType(), True),
    StructField("ELEVATION", DoubleType(), True),
    StructField("NAME", StringType(), True),
    StructField("TEMP", DoubleType(), True),
    StructField("TEMP_ATTRIBUTES", StringType(), True),
    StructField("DEWP", DoubleType(), True),
    StructField("DEWP_ATTRIBUTES", StringType(), True),
    StructField("SLP", DoubleType(), True),
    StructField("SLP_ATTRIBUTES", StringType(), True),
    StructField("STP", DoubleType(), True),
    StructField("STP_ATTRIBUTES", StringType(), True),
    StructField("VISIB", DoubleType(), True),
    StructField("VISIB_ATTRIBUTES", StringType(), True),
    StructField("WDSP", DoubleType(), True),
    StructField("WDSP_ATTRIBUTES", StringType(), True),
    StructField("MXSPD", DoubleType(), True),
    StructField("GUST", DoubleType(), True),
    StructField("MAX", DoubleType(), True),
    StructField("MAX_ATTRIBUTES", StringType(), True),
    StructField("MIN", DoubleType(), True),
    StructField("MIN_ATTRIBUTES", StringType(), True),
    StructField("PRCP", DoubleType(), True),
    StructField("PRCP_ATTRIBUTES", StringType(), True),
    StructField("SNDP", DoubleType(), True),
    StructField("FRSHTT", StringType(), True),
])

In [3]:
def process_year(year, base_path, output_path):
    """
    Process a single year of climate data and save as parquet
    
    Args:
        year: Year to process (e.g., 2010)
        base_path: Base directory containing year folders
        output_path: Directory to save processed parquet files
    """
    print(f"\n{'='*60}")
    print(f"Processing year: {year}")
    print(f"{'='*60}\n")
    
    # Read data for the year
    year_path = f"{base_path}/{year}.tar/*"
    print(f"Reading from: {year_path}")
    
    df = spark.read \
        .option("header", "true") \
        .option("quote", '"') \
        .option("escape", '"') \
        .schema(schema) \
        .csv(year_path)
    
    initial_count = df.count()
    print(f"Initial row count: {initial_count:,}")
    

    # STEP 1: Clean SNDP (Snow Depth)

    print("\n[1/7] Cleaning SNDP (snow depth)...")
    boundary_f = 5 * 9/5 + 32  # 5°C in Fahrenheit ≈ 41°F
    
    df = df.withColumn(
        "SNDP",
        when(
            ((col("SNDP").isNull()) | (col("SNDP") == 999.9) | (col("SNDP") == 9999.9)) & (col("MIN") > boundary_f),
            0
        ).otherwise(col("SNDP"))
    )
    
    # Drop rows where snow is possible but missing
    df = df.filter(
        ~(((col("SNDP").isNull()) | (col("SNDP") == 999.9) | (col("SNDP") == 9999.9)) & (col("MIN") <= boundary_f))
    )
    print(f"   Rows after SNDP cleaning: {df.count():,}")
    

    # STEP 2: Drop unnecessary columns

    print("\n[2/7] Dropping attribute columns and GUST...")
    cols_to_drop = [
        "TEMP_ATTRIBUTES", "DEWP_ATTRIBUTES", "SLP_ATTRIBUTES", "STP_ATTRIBUTES",
        "VISIB_ATTRIBUTES", "WDSP_ATTRIBUTES", "GUST", "MAX_ATTRIBUTES",
        "MIN_ATTRIBUTES", "PRCP_ATTRIBUTES"
    ]
    df = df.drop(*cols_to_drop)
    

    # STEP 3: Drop critical missing values

    print("\n[3/7] Dropping rows with missing critical columns...")
    critical_cols = ["LATITUDE", "LONGITUDE", "ELEVATION", "NAME"]
    df = df.na.drop(subset=critical_cols)
    print(f"   Rows after dropping critical missing: {df.count():,}")
    

    # STEP 4: Impute missing values

    print("\n[4/7] Imputing missing values...")
    numeric_cols = ["DEWP", "SLP", "STP", "VISIB", "WDSP", "MXSPD"]
    
    # Station-wise mean imputation
    print("   - Station-wise mean imputation...")
    for c in numeric_cols:
        window = Window.partitionBy("STATION")
        df = df.withColumn(
            c,
            when(
                (col(c).isNull()) | (col(c) == 999.9) | (col(c) == 9999.9),
                avg(
                    when((col(c) != 999.9) & (col(c) != 9999.9) & col(c).isNotNull(), col(c))
                ).over(window)
            ).otherwise(col(c))
        )
    
    # Grid-wise mean imputation for remaining missing
    print("   - Grid-wise mean imputation...")
    grid_precision = 2.0
    df = df.withColumn("LAT_GRID", spark_round(col("LATITUDE") / grid_precision) * grid_precision) \
           .withColumn("LON_GRID", spark_round(col("LONGITUDE") / grid_precision) * grid_precision)
    
    for c in numeric_cols:
        grid_mean = df.groupBy("LAT_GRID", "LON_GRID").agg(
            avg(
                when((col(c) != 999.9) & (col(c) != 9999.9) & col(c).isNotNull(), col(c))
            ).alias(f"{c}_grid_mean")
        )
        
        df = df.join(grid_mean, on=["LAT_GRID", "LON_GRID"], how="left")
        
        df = df.withColumn(
            c,
            when((col(c).isNull()) | (col(c) == 999.9) | (col(c) == 9999.9), col(f"{c}_grid_mean"))
            .otherwise(col(c))
        ).drop(f"{c}_grid_mean")
    
    df = df.drop("LAT_GRID", "LON_GRID")
    

    # STEP 5: Drop remaining missing values

    print("\n[5/7] Dropping remaining rows with missing values...")
    df = df.dropna()
    
    all_numeric_cols = ["TEMP", "DEWP", "SLP", "STP", "VISIB", "WDSP", "MXSPD", "MAX", "MIN", "PRCP", "SNDP"]
    for c in all_numeric_cols:
        df = df.filter(~col(c).isin([999.9, 9999.9]))
    
    print(f"   Rows after final cleaning: {df.count():,}")
    

    # STEP 6: Data transformations

    print("\n[6/7] Applying data transformations...")
    df = (
        df
        # Convert DATE to datetime
        .withColumn("DATE", to_date(col("DATE"), "yyyy-MM-dd"))
        
        # Extract country from NAME
        .withColumn("COUNTRY", split(col("NAME"), ", ")[size(split(col("NAME"), ", ")) - 1])
        
        # Convert temperatures from °F to °C
        .withColumn("TEMP", (col("TEMP") - 32) * 5/9)
        .withColumn("DEWP", (col("DEWP") - 32) * 5/9)
        .withColumn("MAX", (col("MAX") - 32) * 5/9)
        .withColumn("MIN", (col("MIN") - 32) * 5/9)
        
        # Convert visibility from miles to km
        .withColumn("VISIB", col("VISIB") * 1.60934)
        
        # Convert wind speed from knots to km/h
        .withColumn("WDSP", col("WDSP") * 1.852)
        .withColumn("MXSPD", col("MXSPD") * 1.852)
        
        # Convert precipitation from inches to liters/m²
        .withColumn("PRCP", col("PRCP") * 25.4)
        
        # Convert snow depth from inches to cm
        .withColumn("SNDP", col("SNDP") * 2.54)
    )
    

    # STEP 7: One-hot encode weather events

    print("\n[7/7] One-hot encoding weather events...")
    weather_cols = ["is_Fog", "is_Rain", "is_Snow", "is_Hail", "is_Thunder", "is_Tornado"]
    
    df = df.select(
        "*",
        *[substring(col("FRSHTT"), i + 1, 1).cast("int").alias(w) for i, w in enumerate(weather_cols)]
    ).drop("FRSHTT")
    

    # STEP 8: Save as Parquet

    final_count = df.count()
    print(f"\n{'='*60}")
    print(f"Final row count: {final_count:,}")
    print(f"Rows retained: {100 * final_count / initial_count:.2f}%")
    print(f"{'='*60}\n")
    
    output_file = f"{output_path}/climate_{year}.parquet"
    print(f"Saving to: {output_file}")
    
    df.write.mode("overwrite").parquet(output_file)
    
    print(f"✓ Year {year} successfully processed and saved!\n")
    
    return final_count

In [10]:
base_path = "/home/alumno/reposirotio/Grupo3"
output_path = "processed_data/"

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# TEST: Process only 2010 first
print("\n" + "="*60)
print("TESTING WITH YEAR 2010")
print("="*60)

try:
    print("\n" + "="*60)
    print("PROCESSING ALL YEARS (2010-2025)")
    print("="*60)
    
    total_rows = 0
    for year in range(2011, 2025): # From 2010 to 2024  
        try:
            rows = process_year(year, base_path, output_path)
            total_rows += rows
        except Exception as e:
            print(f"\nERROR processing year {year}: {str(e)}")
            continue
    
    print("\n" + "="*60)
    print(f"ALL YEARS PROCESSED!")
    print(f"Total rows processed: {total_rows:,}")
    print("="*60)

except Exception as e:
        print(f"\nERROR: {str(e)}")
        import traceback
        traceback.print_exc()

spark.stop()


TESTING WITH YEAR 2010

PROCESSING ALL YEARS (2010-2025)

Processing year: 2011

Reading from: /home/alumno/reposirotio/Grupo3/2011.tar/*


25/12/31 13:06:57 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: /home/alumno/reposirotio/Grupo3/2011.tar/*.
java.io.FileNotFoundException: File /home/alumno/reposirotio/Grupo3/2011.tar/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.ap

Initial row count: 7,382,831

[1/7] Cleaning SNDP (snow depth)...


   Rows after SNDP cleaning: 4,920,926

[2/7] Dropping attribute columns and GUST...

[3/7] Dropping rows with missing critical columns...


   Rows after dropping critical missing: 4,917,070

[4/7] Imputing missing values...
   - Station-wise mean imputation...
   - Grid-wise mean imputation...

[5/7] Dropping remaining rows with missing values...


...

[11317,552s][warning][gc,alloc] Executor task launch worker for task 1.0 in stage 1951.0 (TID 269111): Retried waiting for GCLocker too often allocating 524288 words


25/12/31 15:45:11 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.



Final row count: 4,956,096
Rows retained: 62.93%

Saving to: processed_data//climate_2024.parquet


25/12/31 15:45:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 01001099999.csv                                                                                     0100644 0000000 0000000 00000244171 14744005554 011521  0                                                                                                    ustar 00                                                                0000000 0000000                                                                                                                                                                        "STATION", DATE, LATITUDE, LONGITUDE, ELEVATION, NAME, TEMP, DEWP, SLP, STP, VISIB, WDSP, MXSPD, MAX, MIN, PRCP, SNDP, FRSHTT
 Schema: STATION, DATE, LATITUDE, LONGITUDE, ELEVATION, NAME, TEMP, DEWP, SLP, STP, VISIB, WDSP, MXSPD, MAX, MIN, PRCP, SNDP, FRSHTT
Expected: STATION but found: 01001099999.csv                                                                                     0100644

✓ Year 2024 successfully processed and saved!


ALL YEARS PROCESSED!
Total rows processed: 66,714,860
